In [ ]:
from ortools.sat.python import cp_model
def solver(N_classes, N_rooms, t, g, s,c):
    # Khởi tạo model
    model = cp_model.CpModel()
    # Khai báo các biến
    #N_classes = 10  # Số lượng lớp học
    #N_rooms = 2   # Số lượng phòng học
    #N_slots = 60     # Số lượng tiết học
    #t = [4,4,4,2,4,3,2,3,4,3]          # Thời lượng của từng lớp học 
    #g = [1,1,1,2,2,1,2,2,1,1]          # Giáo viên của từng lớp học 

    # Biến X_i, Y_i
    X = [model.NewIntVar(1, 60, f'X_{i}') for i in range(N_classes)]
    Y = [[model.NewBoolVar(f'Y_{i}_{j}')for j in range(N_rooms)] for i in range(N_classes)]
    # Ràng buộc: Trong một phòng học trong một tiết chỉ có một lớp học.
    a = model.NewBoolVar("a")
    for i1 in range(N_classes):
        for i2 in range(i1 + 1, N_classes):
            for j in range(N_rooms):
                model.Add(X[i1] + t[i1] <= X[i2]).OnlyEnforceIf([Y[i1][j],Y[i2][j],a])
                model.Add(X[i2] + t[i2] <= X[i1]).OnlyEnforceIf([Y[i1][j],Y[i2][j],a.Not()])
    b = model.NewBoolVar("b")
    # Ràng buộc: Trong một tiết, một giáo viên chỉ dạy một lớp.
    for i1 in range(N_classes):
        for i2 in range(i1 + 1, N_classes):
            if g[i1] == g[i2]:
                model.Add(X[i1] + t[i1] <= X[i2]).OnlyEnforceIf(b)
                model.Add(X[i2] + t[i2] <= X[i1]).OnlyEnforceIf(b.Not()) 
    # Ràng buộc: Thời điểm kết thúc của lớp học không được vượt quá tiết 60
    for i in range(N_classes):
        model.Add(X[i]+t[i]-1<=60)
    #Ràng buộc: 1 lớp học chỉ có thể diễn ra trong tối đa 1 phòng học
    for i in range(N_classes):
        model.Add(sum(Y[i][j] for j in range(N_rooms))<=1)

    valid_ranges = [(1, 6), (7, 12), (13, 18), (19, 24), (25, 30), 
                    (31, 36), (37, 42), (43, 48), (49, 54), (55, 60)]

    #Ràng buộc: Mỗi lớp học chỉ nên diễn ra trong 1 buổi có 6 tiết
    bool_vars = []
    for i in range(N_classes):
        bool_var_ranges = []
        for (start, end) in valid_ranges:
            bool_var = model.NewBoolVar(f'b_{i}_{start}_{end}')
            bool_var_ranges.append(bool_var)
            model.Add(X[i] >= start).OnlyEnforceIf(bool_var)
            model.Add(X[i]+t[i]-1 <= end).OnlyEnforceIf(bool_var)
        bool_vars.append(bool_var_ranges)

    # Đảm bảo rằng mỗi X_i phải nằm trong ít nhất một khoảng thời gian hợp lệ
    for i in range(N_classes):
        model.AddBoolOr(bool_vars[i])

    # Ràng buộc: Giới hạn phòng học
    #s = [20,20]  # Sức chứa của từng phòng học
    #c = [15,18,15,18,11,15,27,18,13,10]  # Số lượng học sinh của từng lớp
    for i in range(N_classes):
        for j in range(N_rooms):
            model.Add(s[j] >= c[i]).OnlyEnforceIf(Y[i][j])
    Z = sum(Y[i][j] for j in range(N_rooms) for i in range(N_classes))
    # Hàm mục tiêu
    model.Maximize(Z)

    # Tạo solver và giải bài toán
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Kiểm tra kết quả và in ra giá trị của các biến
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print(solver.Value(Z))
        for i in range(N_classes):
            for j in range(N_rooms):
                if solver.Value(Y[i][j]):
                    print(f'{i+1} {solver.Value(X[i])} {j+1}')
    else:
        print('Không tìm thấy giải pháp khả thi.')
line = input()
N_classes, N_rooms = line.split()
N_classes = int(N_classes)
N_rooms = int(N_rooms)
t = [0 for _ in range(N_classes)]
g = [0 for _ in range(N_classes)]
c = [0 for _ in range(N_classes)]
s = [0 for _ in range(N_rooms)]
for i in range(N_classes):
    line = input()
    t[i], g[i], c[i] = line.split()
    t[i] = int(t[i])
    g[i] = int(g[i])
    c[i] = int(c[i])
line = input()
s =  [int(cap) for cap in line.split()]
solver(N_classes, N_rooms, t,g,s,c)